In [1]:
# Version 2 - Measures performance (precision,recall,f-measure), and plots words wrt SVM coefficients 

In [ ]:
###############################################################################
### packages required to run code.  Make sure to install all required packages.
###############################################################################
import re,string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd
import os

from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix


In [ ]:
#Section 3, 4, 5: Classifiers
import numpy as np
from sklearn.svm import SVC
from matplotlib.pyplot import figure


def classifiers(x, y, model_type,class_names,feature_names):
    
    #this function is to fit 3 different model scenarios.  Support vector machines, logistic regressions, naive bayes.
    #svm = Support vector machin
    #logistic = Logistic regression
    #naive_bayes = Naive Bayes Multinomial
    
 
    #function returns the train test split scores of each model.
        
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    

    if model_type == 'svm':
        print("svm")
        clf = SVC(kernel='linear')
        clf.fit(X_train, y_train)
        pd.Series(abs(clf.coef_[0]), index=feature_names).nlargest(20).plot(kind='barh')
       

    elif model_type == 'logistic':
        print("logistic")
        clf = LogisticRegression()
        clf.fit(X_train, y_train)
    elif model_type == 'naive_bayes':
        print("naive_bayes")
        clf = MultinomialNB()
        clf.fit(X_train, y_train)
    

    
    #NOTE THIS IS AN OVERFIT CASE SO YOU KNOW THAT IT IS WORKING
    #ONCE YOU GET CLOSE TO 100% ACCURACY THEN COMMENT OUT THESE TWO LINES
    #predictions = model.predict(X_train)
    #accy = accuracy_score(y_train, predictions)
    
    #NOTE THIS IS THE CORRECT WAY TO TEST YOUR CLASSIFIERS
    #UNCOMMENT THESE TWO LINES ONCE THE OVERFIT CASE YIELDS CLOSE TO 100% ACCURACY
    predictions = clf.predict(X_test)
    accy = accuracy_score(y_test, predictions)
    cm=confusion_matrix(y_test, predictions)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    titles_options = [("Confusion matrix, without normalization", None),
                      ("Normalized confusion matrix", 'true')]
    for title, normalize in titles_options:
        disp = plot_confusion_matrix(clf, X_test, y_test,
                                     display_labels=class_names,
                                     cmap=plt.cm.Blues,
                                     normalize=normalize)
        disp.ax_.set_title(title)

        print(title)
        print(disp.confusion_matrix)

    plt.show()
    
    return accy,cm,X_test,y_test,predictions

In [ ]:

dir='/media/jsleeman/extradrive1/dev/teach/nlp_northwestern/sentiment_data/txt_sentoken/'
movie_data = load_files(dir)
X, y = movie_data.data, movie_data.target
class_names=set(y)
documents = []
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_ = tfidfconverter.fit_transform(documents).toarray()

words = tfidfconverter.get_feature_names()

In [ ]:
result,cm,X_test,y_test,predictions=classifiers(X_,y,'svm',class_names,words)
print(result)

In [ ]:
misclassified = np.where(y_test != predictions)
print(misclassified)

In [ ]:
txt_lookup={}
for i in range(0,len(documents)):
    txt_lookup[str(X_[i])]=documents[i]
#creating datafram from TFIDF Matrix
print(txt_lookup[str(X_test[0])])


In [ ]:
print(predictions)
from nltk.metrics import precision,recall,f_measure
import collections
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
for i, label in enumerate(y_test):
    refsets[label].add(i)
    observed = predictions[i]
    testsets[observed].add(i)

print(refsets)
print('pos precision:', precision(refsets[1], testsets[1]))
print( 'pos recall:', recall(refsets[1], testsets[1]))
print('pos F-measure:', f_measure(refsets[1], testsets[1]))
print('neg precision:', precision(refsets[0], testsets[0]))
print('neg recall:', recall(refsets[0], testsets[0]))
print('neg F-measure:', f_measure(refsets[0], testsets[0]))


In [ ]:
result,cm,X_test,y_test,predictions=classifiers(X_,y,'logistic',class_names,words)

In [ ]:
result,cm,X_test,y_test,predictions=classifiers(X_,y,'naive_bayes',class_names,words)

In [ ]:

# Instantiate and train the classifier

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=1)
X_train, X_test, y_train, y_test = train_test_split(X_,y, test_size=0.33, random_state=42)
clf.fit(X_train, y_train) 


# Check the results using metrics
from sklearn import metrics
y_pred = clf.predict(X_test)
accy = accuracy_score(y_test, y_pred)
print(accy)
print(metrics.confusion_matrix(y_pred, y_test))